In [2]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

# Summer  Gold  Silver  Bronze  Total  # Winter  Gold.1  \
Afghanistan        13     0       0       2      2         0       0   
Algeria            12     5       2       8     15         3       0   
Argentina          23    18      24      28     70        18       0   
Armenia             5     1       2       9     12         6       0   
Australasia         2     3       4       5     12         0       0   

             Silver.1  Bronze.1  Total.1  # Games  Gold.2  Silver.2  Bronze.2  \
Afghanistan         0         0        0       13       0         0         2   
Algeria             0         0        0       15       5         2         8   
Argentina           0         0        0       41      18        24        28   
Armenia             0         0        0       11       1         2         9   
Australasia         0         0        0        2       3         4         5   

             Combined total   ID  
Afghanistan               2  AFG  
Algeria                  15  ALG  
Argentina                70  ARG  
Armenia                  12  ARM  
Australasia              12  ANZ

In [3]:
# Which country has won the most gold medals in summer games?

def answer_one():
    return df[(df['Gold'] == max(df['Gold']))].index.values[0]

# answer_one()

In [4]:
# Which country had the biggest difference between their summer and winter gold medal counts?

def answer_two():
    max_difference = max(abs(df['Gold'] - df['Gold.1']))
    return df[(df['Gold'] - df['Gold.1']) == max_difference].index.values[0]

# answer_two()

In [5]:
# Which country has the biggest difference between their summer gold medal counts and winter 
# gold medal counts relative to their total gold medal count?
# Only include countries that have won at least 1 gold in both summer and winter.

def answer_three():
    new_df = df[(df['Gold.1'] > 0) & (df['Gold.1'] > 0)]
    max_rel_diff = max((new_df['Gold'] - new_df['Gold.1'])/new_df['Gold.2'])
    return new_df[((new_df['Gold'] - new_df['Gold.1'])/new_df['Gold.2']) == max_rel_diff].index.values[0]
    
# answer_three()

In [6]:
# Write a function that creates a Series called "Points" which is a weighted value where
# each gold medal (Gold.2) counts for 3 points, 
# silver medals (Silver.2) for 2 points, and bronze medals (Bronze.2) for 1 point.
# The function should return only the column (a Series object) which you created, with the country names as indices.

def answer_four():
    cpy_df = df.copy()
    return (cpy_df['Gold.2'] * 3) + (cpy_df['Silver.2'] * 2) + (cpy_df['Bronze.2'])

# answer_four()

In [7]:
census_df = pd.read_csv('census.csv')
census_df.head()

SUMLEV  REGION  DIVISION  STATE  COUNTY   STNAME         CTYNAME  \
0      40       3         6      1       0  Alabama         Alabama   
1      50       3         6      1       1  Alabama  Autauga County   
2      50       3         6      1       3  Alabama  Baldwin County   
3      50       3         6      1       5  Alabama  Barbour County   
4      50       3         6      1       7  Alabama     Bibb County   

   CENSUS2010POP  ESTIMATESBASE2010  POPESTIMATE2010     ...       \
0        4779736            4780127          4785161     ...        
1          54571              54571            54660     ...        
2         182265             182265           183193     ...        
3          27457              27457            27341     ...        
4          22915              22919            22861     ...        

   RDOMESTICMIG2011  RDOMESTICMIG2012  RDOMESTICMIG2013  RDOMESTICMIG2014  \
0          0.002295         -0.193196          0.381066          0.582002   
1          7.242091         -2.915927         -3.012349          2.265971   
2         14.832960         17.647293         21.845705         19.243287   
3         -4.728132         -2.500690         -7.056824         -3.904217   
4         -5.527043         -5.068871         -6.201001         -0.177537   

   RDOMESTICMIG2015  RNETMIG2011  RNETMIG2012  RNETMIG2013  RNETMIG2014  \
0         -0.467369     1.030015     0.826644     1.383282     1.724718   
1         -2.530799     7.606016    -2.626146    -2.722002     2.592270   
2         17.197872    15.844176    18.559627    22.727626    20.317142   
3        -10.543299    -4.874741    -2.758113    -7.167664    -3.978583   
4          0.177258    -5.088389    -4.363636    -5.403729     0.754533   

   RNETMIG2015  
0     0.712594  
1    -2.187333  
2    18.293499  
3   -10.543299  
4     1.107861  

[5 rows x 100 columns]

In [8]:
# Which state has the most counties in it?

def answer_five():
    cdf = census_df[census_df['SUMLEV'] != 40]
    states = census_df['STNAME'].unique()
    max_count = 0
    state_name = ''
    for st in states:
        count = len(cdf[cdf['STNAME'] == st])
        if count > max_count:
            max_count = count
            state_name = st
    return state_name

# answer_five()

'Texas'

In [81]:
# Only looking at the three most populous counties for each state,
# what are the three most populous states (in order of highest population to lowest population)? 
# Use CENSUS2010POP.

def answer_six():
    states = census_df['STNAME'].unique()
    cdf = census_df[census_df['SUMLEV'] != 40]
    cdf = cdf[['STNAME','CTYNAME','CENSUS2010POP']]
    top_3_pop = []
    for st in states:
        top_3 = cdf[cdf['STNAME'] == st].sort_values('CENSUS2010POP', ascending = False)[:3]
        pop_sum = sum(top_3['CENSUS2010POP'])
        top_3_pop.append(pop_sum)
    pop_df = pd.DataFrame(top_3_pop, index = states, columns = ['CENSUS2010POP'])
    pop_df = pop_df.sort_values('CENSUS2010POP', ascending = False)
    return list(pop_df.index)[:3]

# answer_six()

['California', 'Texas', 'Illinois']

In [79]:
cdf = census_df[census_df['SUMLEV'] != 40]
counties = cdf['CTYNAME']
diff1011 = abs(cdf['POPESTIMATE2010'] - cdf['POPESTIMATE2011'])
diff1112 = abs(cdf['POPESTIMATE2011'] - cdf['POPESTIMATE2012'])
diff1213 = abs(cdf['POPESTIMATE2012'] - cdf['POPESTIMATE2013'])
diff1314 = abs(cdf['POPESTIMATE2013'] - cdf['POPESTIMATE2014'])
diff1415 = abs(cdf['POPESTIMATE2014'] - cdf['POPESTIMATE2015'])
diff_df = pd.DataFrame()
diff_df['CTYNAME'] = counties
diff_df['10-11'] = diff1011
diff_df['11-12'] = diff1112
diff_df['12-13'] = diff1213
diff_df['13-14'] = diff1314
diff_df['14-15'] = diff1415
tdf = diff_df.T
tdf.values

array([['Autauga County', 'Baldwin County', 'Barbour County', ...,
        'Uinta County', 'Washakie County', 'Weston County'],
       [593, 3466, 115, ..., 190, 76, 67],
       [78, 3737, 67, ..., 77, 26, 49],
       [137, 4730, 186, ..., 33, 0, 95],
       [252, 4587, 158, ..., 119, 127, 25],
       [57, 3996, 326, ..., 81, 12, 49]], dtype=object)

In [63]:
# In this datafile, the United States is broken up into four regions using the "REGION" column.
# Create a query that finds the counties that belong to regions 1 or 2,
# whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.
# This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] 
# and the same index ID as the census_df (sorted ascending by index).

def answer_eight():
    cond_1 = (census_df['SUMLEV'] != 40)
    cond_2 = (census_df['CTYNAME'] == 'Washington County')
    cond_3 = (census_df['POPESTIMATE2015'] > census_df['POPESTIMATE2014'])
    cond_4 = (census_df['REGION'] <= 2)
    return census_df[cond_1 & cond_2 & cond_3 & cond_4][['STNAME', 'CTYNAME']]

# answer_eight()

STNAME            CTYNAME
896           Iowa  Washington County
1419     Minnesota  Washington County
2345  Pennsylvania  Washington County
2355  Rhode Island  Washington County
3163     Wisconsin  Washington County